## Part 1: Preprocessing

In [45]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [25]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [26]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]

In [27]:
# Create a list of at least 10 column names to use as X data
X = ['Age', 'BusinessTravel', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
             'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MaritalStatus', 'NumCompaniesWorked',
             'PercentSalaryHike', 'TotalWorkingYears', 'WorkLifeBalance', 'YearsAtCompany']


# Create X_df using your selected columns
X_df = attrition_df[X]

# Show the data types for X_df
X_df.dtypes


,0
Age,int64
BusinessTravel,object
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
JobInvolvement,int64
JobLevel,int64
JobSatisfaction,int64
MaritalStatus,object
NumCompaniesWorked,int64


In [28]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)



In [29]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

In [30]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [31]:
# Create a OneHotEncoder for the Department column
dept_ohe = OneHotEncoder()

# Fit the encoder to the training data
dept_ohe.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoded = dept_ohe.transform(y_train[['Department']])
y_test_dept_encoded = dept_ohe.transform(y_test[['Department']])



In [40]:
# Create a OneHotEncoder for the Attrition column
attrition_ohe = OneHotEncoder()

# Fit the encoder to the training data
attrition_ohe.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_ohe.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_ohe.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [33]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [34]:
# Create a branch for Department
# with a hidden layer and an output layer
dept_hidden = layers.Dense(16, activation='relu')(shared_layer2)
dept_output = layers.Dense(3, activation='softmax', name='department')(dept_hidden)

# Create the hidden layer
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer2)


# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition')(attrition_hidden)


In [35]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer2)
attrition_output = layers.Dense(2, activation='softmax', name='attrition')(attrition_hidden)

# Create the hidden layer
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer2)


# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition')(attrition_hidden)


In [36]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department': 'categorical_crossentropy', 'attrition': 'categorical_crossentropy'},
              metrics={'department': 'accuracy', 'attrition': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 14)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 64)             │            960 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 32)             │          2,080 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 16)             │            528 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 16)             │            528 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department (Dense)        │ (None, 3)              │             51 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition (Dense)         │ (None, 2)              │             34 │ dense_11[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,181 (16.33 KB)

 Trainable params: 4,181 (16.33 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
# Train the model
history = model.fit(X_train_scaled,
                    {'department': y_train_dept_encoded, 'attrition': y_train_attrition_encoded},
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=0)

TypeError: Failed to convert elements of SparseTensor(indices=Tensor("data_1:0", shape=(None, 2), dtype=int64), values=Tensor("data_2:0", shape=(None,), dtype=float32), dense_shape=Tensor("data_3:0", shape=(2,), dtype=int64)) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.

In [41]:
# Evaluate the model with the testing data
test_loss, dept_loss, attrition_loss, dept_acc, attrition_acc = model.evaluate(X_test_scaled,
                                                                               [y_test_dept_encoded, y_test_attrition_encoded],
                                                                               verbose=0)

TypeError: Failed to convert elements of SparseTensor(indices=Tensor("DeserializeSparse:0", shape=(None, 2), dtype=int64), values=Tensor("DeserializeSparse:1", shape=(None,), dtype=float32), dense_shape=Tensor("stack:0", shape=(2,), dtype=int64)) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.

In [ ]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {dept_acc:.4f}")
print(f"Attrition predictions accuracy: {attrition_acc:.4f}")

Department predictions accuracy: 0.5271739363670349
Attrition predictions accuracy: 0.8260869383811951


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. no - in this case we are looking for outliers; with much fewer attrictian data points, accuracy is misleading
2. for the binary classifier (attrician or not), I used sigmoid to return a likelihood closer to 0 or 1; for the department, I used the softmax because of the multiclass prediction
3. selecting better features (feature engineering), more training data